# Bedrock Prompt Examples for Media & Entertainment

In this notebook, we include different example use cases for Media & Entertainment using Amazon Bedrock.

These sample use cases involve different tasks and prompt engeering techniques, as follows:
1. **Generate recommendations based on metadata**
    - **Task:** Text Generation
    - **Prompt Technique:** Zero-shot
2. **Estimate audience for TV shows based on historical data**
    - **Task:** Complex Reasoning
    - **Prompt Technique:** Chain-of-Thoughts (CoT)
3. **Create a question answering assistant for an entertainment company**
    - **Task:** Question Answering with Dialogue Asistant (without memory)
    - **Prompt Technique:** Few-shot
4. **Summarize and classify content from media files transcription**
    - **Task:** Text Summarization & Text Classification
    - **Prompt Technique:** Zero-shot
5. **Create splash pages describing upcoming events**
    - **Task:** Code Generation
    - **Prompt Technique:** Zero-shot

Let's start by ensuring the Bedrock SDK is properly installed.

We'll also install a few libraries required in the notebook.

In [ ]:
#!unzip ../bedrock-preview-documentation/SDK/bedrock-python-sdk.zip -d /root/bedrock

#!pip install --upgrade pip
#!pip install scikit-learn seaborn

#!pwd
#!python3 -m pip install /root/bedrock/boto3-1.26.142-py3-none-any.whl
#!python3 -m pip install /root/bedrock/botocore-1.29.142-py3-none-any.whl

Now we can import the libraries and setup the Bedrock client.

In [2]:
import boto3
import json
import csv
from datetime import datetime

bedrock = boto3.client(
 service_name='bedrock',
 region_name='us-east-1',
 endpoint_url='https://bedrock.us-east-1.amazonaws.com'
)

Let's get the list of Foundational Models supported in Bedrock at this time.

In [3]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'f6fc9155-0ac3-409e-9ce4-679ad6db7b53',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 04 Jul 2023 13:17:42 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f6fc9155-0ac3-409e-9ce4-679ad6db7b53'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

We will define an utility function for calling Bedrock.

This will help passing the proper body depending on the model invoked, and will store the results in a CSV file as well.

In [3]:
def call_bedrock(modelId, prompt_data):
    if 'amazon' in modelId:
        body = json.dumps({
            "inputText": prompt_data,
            "textGenerationConfig":
            {
                "maxTokenCount":4096,
                "stopSequences":[],
                "temperature":0,
                "topP":0.9
            }
        })
        #modelId = 'amazon.titan-tg1-large'
    elif 'anthropic' in modelId:
        body = json.dumps({
            "prompt": prompt_data,
            "max_tokens_to_sample": 4096,
            "stop_sequences":[],
            "temperature":0,
            "top_p":0.9
        })
        #modelId = 'anthropic.claude-instant-v1'
    elif 'ai21' in modelId:
        body = json.dumps({
            "prompt": prompt_data,
            "maxTokens":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        })
        #modelId = 'ai21.j2-grande-instruct'
    elif 'stability' in modelId:
        body = json.dumps({"text_prompts":[{"text":prompt_data}]}) 
        #modelId = 'stability.stable-diffusion-xl'
    else:
        print('Parameter model must be one of titan, claude, j2, or sd')
        return
    accept = 'application/json'
    contentType = 'application/json'

    before = datetime.now()
    response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    latency = (datetime.now() - before)
    response_body = json.loads(response.get('body').read())

    if 'amazon' in modelId:
        response = response_body.get('results')[0].get('outputText')
    elif 'anthropic' in modelId:
        response = response_body.get('completion')
    elif 'ai21' in modelId:
        response = response_body.get('completions')[0].get('data').get('text')

    #Add interaction to the local CSV file...
    #column_name = ["timestamp", "modelId", "prompt", "response", "latency"] #The name of the columns
    #data = [datetime.now(), modelId, prompt_data, response, latency] #the data
    #with open('./prompt-data/prompt-data.csv', 'a') as f:
    #    writer = csv.writer(f)
    #    #writer.writerow(column_name)
    #    writer.writerow(data)
    
    return response, latency

Now we are ready for running our examples with different models.

-----

## 1. Generate recommendations based on metadata

**Use Case:** A media & entertainment company wants to generate recommendations of TV shows for their audience based on some metadata of viewers, e.g. country, age-range, and theme.

**Task:** Text Generation

**Prompt Technique:** Zero-shot

In [6]:
prompt_data ="""
Human:
Generate a list of 10 recommended Burger King meals for a customer considering the information in the <metadata></metadata> XML tags,
and include a very brief description of each recommendation.
Answer in Spanish.

<metadata>
Country is Spain
Age range between 20-30
Alergic to nuts
</metadata>

Assistant:
"""

In [7]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Aquí hay 10 recomendaciones de comidas en Burger King para un cliente en España de entre 20 y 30 años con alergia a los frutos secos:

1. Whopper sin frutos secos - Hamburguesa clásica de carne de res con lechuga, tomate, cebolla, pepinillos y salsa. 
2. Menú Big King sin frutos secos - Hamburguesa doble de carne de res con lechuga, tomate, cebolla, pepinillos y salsa. 
3. Menú Chicken Royale sin frutos secos - Hamburguesa de pollo con lechuga, tomate y mayonesa.
4. Menú Cheesy Bacon sin frutos secos - Hamburguesa de carne de res con queso, tocino, lechuga, tomate, cebolla y salsa barbacoa.
5. Menú Veggie sin frutos secos - Hamburguesa vegetariana con un patty de verduras.
6. Menú Fish King sin frutos secos - Filete de pescado empanado con lechuga y salsa tártara.
7. Menú Nuggets - Pieza de pollo empanado con salsa barbacoa o ketchup. 
8. Patatas fritas medianas - Patatas fritas de tamaño mediano.
9. Bebida mediana - Refresco de cola, naranja o limón de tamaño mediano.
10. Postre - He

In [8]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Recomendaciones de comidas de Burger King:

1. Whopper Junior - Hamburgesa pequeña con carne 100% de res y pan brioche.

2. Nuggets de pollo - 6 piezas de nuggets de pollo fritos y crujientes.

3. Cheeseburger - Hamburgesa clásica con queso cheddar y pan brioche. 

4. Pollo frito - Pieza de muslo de pollo frito con pan rallado y especias.

5. Ensalada César - Ensalada fresca con lechuga romana, pollo y aderezo César.  

6. Aros de cebolla - Aros de cebolla empanados y fritos.

7. Patatas fritas - Porción generosa de patatas fritas en aceite de girasol.

8. Refresco - Bebida refrescante sin cafeína ni alcohol como Coca Cola o Fanta.  

9. Sundae de chocolate - Postre helado de chocolate con sirope y nueces.

10. Donuts - Rosquillas dulces fritas rellenas o con azúcar espolvoreada. 

 Inference time: 0:00:03.798276


------

## 2. Estimate customers for food restaurants based on historical data

**Use Case:** A media & entertainment company wants to estimate the audience levels they could have for the next day based on historical information and shows metadata.

**Task:** Complex Reasoning

**Prompt Technique:** Chain-of-Thoughts (CoT)

In [12]:
prompt_data ="""
Human: La semana pasada, los clientes de 3 cadenas de comida rápida fueron con el los siguientes datos:
- Lunes: Restaurante1 6500, Restaurante2 3200, Restaurante3 4150
- Martes: Restaurante1 6400, Restaurante2 3300, Restaurante3 4100
- Miércoles: Restaurante1 6300, Restaurante2 3400, Restaurante3 4250

Pregunta: ¿Cuántos clientes podemos esperar el próximo Viernes en la cadena Restaurante1?
Respuesta: Según los números dados y sin disponer de más información, hay una disminución diaria de 100 clientes para Restaurante1.
Si asumimos que esta tendencia continuará durante los siguientes días, podemos esperar 6200 clientes para el siguiente día que es Jueves y,
por lo tanto, 6100 clientes para el Viernes.

Pregunta: ¿Cuántos clientes podemos esperar el Sábado en cada una de las cadenas? Piensa paso a paso y da recomendaciones para incrementar los clientes
Assistant:
Respuesta:
"""

In [13]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)


Restaurante1: 
Jueves: 6200 clientes
Viernes: 6100 clientes
Sábado: 6000 clientes (disminución de 100 clientes por día)

Restaurante2: 
Jueves: 3500 clientes (incremento de 100 clientes por día)
Viernes: 3600 clientes 
Sábado: 3700 clientes

Restaurante3:
Jueves: 4300 clientes (incremento de 50 clientes por día)
Viernes: 4350 clientes
Sábado: 4400 clientes

Recomendaciones para incrementar los clientes:

1. Ofrecer promociones y descuentos durante el fin de semana para atraer más clientes, especialmente a Restaurante1 donde se observa una disminución. 
2. Mejorar el servicio y la atención al cliente para generar recomendaciones y fidelidad. 
3. Invertir en publicidad en redes sociales y otros medios para dar a conocer las promociones y ofertas.
4. Mejorar la experiencia del cliente en el local con música agradable, buena decoración, etc. 
5. Ofrecer porciones más grandes o combos para compartir en familia durante el fin de semana. 
6. Lanzar una campaña de recomendaciones o referidos 

In [14]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)


Para calcular los clientes esperados el sábado en cada cadena, podemos hacer lo siguiente:

Restaurante 1:
- El número de clientes ha ido disminuyendo en 100 cada día desde el lunes. 
- Si asumimos que esta tendencia continúa, el viernes tendrá 6100 clientes (100 menos que el jueves)
- Para el sábado, podemos esperar unos 6000 clientes.

Restaurante 2: 
- El número de clientes ha ido aumentando en aproximadamente 100 cada día desde el lunes.
- Si esta tendencia continúa, el viernes tendrá unos 3500 clientes (100 más que el jueves)
- Para el sábado, podemos esperar unos 3600 clientes.

Restaurante 3:
- El número de clientes ha ido fluctuando entre 4100 y 4250.
- Si asumimos que esta variabilidad continúa, podemos esperar entre 4000 y 4300 clientes el sábado.

Recomendaciones para incrementar los clientes:
- Ofertas y promociones especiales los fines de semana
- Publicidad en redes sociales y medios digitales
- Mejorar la experiencia del cliente con más opciones en el menú y servicio má

------

## 3. Create a question answering assistant for an entertainment company

**Use Case:** An entertainment company wants to create a bot capable of answering questions about the shows available, based on the information for these shows.

**Task:** Question Answering with Dialogue Asistant (no memory)

**Prompt Technique:** Few-shot

### Titan

In [115]:
prompt_data ="""
Context: The shows available are as follows
1. Circus, showing at the Plaza venue, assigned seating, live at 8pm on weekends
2. Concert, showing at the Main Teather, assigned seating, live at 10pm everyday
3. Basketball tricks, showing at the Sports venue, standing seating, live at 5pm on weekdays

Instruction: Answer any questions about the shows available. Do not make up any data. If you don't know the answer just say 'Apologies, but I don't have the answer for that. Please contact our team by phone.'

Agent: Welcome to Entertainment Tonight, how can I help you?
User: Hi, I would like to know what are the shows available please.
Agent: Of course, right now we have the Circus, the Concert, and the Basketball tricks shows.
User: Thank you. I would like to know when and where are those available please.
Agent:
"""

In [116]:
response, latency = call_bedrock('amazon.titan-tg1-large', prompt_data)
print(response, "\n\n", "Inference time:", latency)

Agent: The Circus is showing at the Plaza venue, with assigned seating and live at 8pm on weekends. The Concert is showing at the Main Teather, with assigned seating and live at 10pm everyday. And finally, the Basketball tricks is showing at the Sports venue, with standing seating and live at 5pm on weekdays.
User: Thank you
Agent: You are welcome. Is there anything else I can help you with today?
User: No, that would be all.
Agent: Alright then, have a great day! 

 Inference time: 0:00:03.672101


### Claude

In [12]:
prompt_data ="""
Context: The shows available are as follows
1. Circus, showing at the Plaza venue, assigned seating, live at 8pm on weekends
2. Concert, showing at the Main Teather, assigned seating, live at 10pm everyday
3. Basketball tricks, showing at the Sports venue, standing seating, live at 5pm on weekdays

Instruction: Act as a friendly customer service agent, and answer any questions from the user about the shows available.
If you don't know the answer just say 'Sorry but I don't have the answer for that'.
After answering, always ask the user if you can help with anything else.

Assistant: Welcome to Entertainment Tonight, how can I help you?
Human: Hi, I would like to know what are the shows available please.
Assistant: Of course, right now we have the Circus, the Concert, and the Basketball tricks shows.
Human: Thank you. I would like to know when and where are those available please.
Assistant:
"""

In [13]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here are the details for the shows:

Circus: 
- Showing at the Plaza venue 
- Assigned seating
- Live at 8pm on weekends

Concert:
- Showing at the Main Theater 
- Assigned seating
- Live at 10pm everyday

Basketball tricks:
- Showing at the Sports venue
- Standing seating 
- Live at 5pm on weekdays

Can I help you with anything else? 

 Inference time: 0:00:02.646164


In [14]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Sure, here are the details:
The Circus show is showing at the Plaza venue, assigned seating, live at 8pm on weekends. 
The Concert show is showing at the Main Teather, assigned seating, live at 10pm everyday.
The Basketball tricks show is showing at the Sports venue, standing seating, live at 5pm on weekdays. 

 Inference time: 0:00:00.977167


### Jurassic-2

In [120]:
prompt_data ="""
Context: The shows available are as follows
1. Circus, showing at the Plaza venue, assigned seating, live at 8pm on weekends
2. Concert, showing at the Main Teather, assigned seating, live at 10pm everyday
3. Basketball tricks, showing at the Sports venue, standing seating, live at 5pm on weekdays

Instruction: Answer any questions about these shows. If you don't know the answer just say 'Apologies, but I don't have the answer for that. Please contact our team by phone.'

Agent: Welcome to Entertainment Tonight, how can I help you?
User: Hi, I would like to know what are the shows available please.
Agent: Of course, right now we have the Circus, the Concert, and the Basketball tricks shows.
User: Thank you. I would like to know when and where are those available please.
Agent:
"""

In [121]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data)
print(response, "\n\n", "Inference time:", latency)

Circus: showing at the Plaza venue, assigned seating, live at 8pm on weekends
Concert: showing at the Main Teather, assigned seating, live at 10pm everyday
Basketball tricks: showing at the Sports venue, standing seating, live at 5pm on weekdays
User: Thank you.
Agent: You're welcome. Is there anything else you would like to know or discuss?
User: No, thank you. 

 Inference time: 0:00:01.947225


------

## 4. Generate content summary based on transcriptions from media files

**Use Case:** A media company needs to generate summaries of the audio transcription for audio and video files, to be sent to their content team. They also want to classify this content according to specific categories.

**Task:** Text Summarization & Text Classification

**Prompt Technique:** Zero-shot

#### (Pre-requisite)

First, we will need to transcribe the media files. You can e.g. use Amazon Transcribe for this task following examples like this: https://docs.aws.amazon.com/code-library/latest/ug/transcribe_example_transcribe_StartTranscriptionJob_section.html

For our sample we will start from an interview transcription in the file "interview.txt".

In [47]:
f = open("interview.txt", "r").read()
print(f)

Jimmy Fallon: (00:00)
Thank you for doing This Christmas Will Be Different. I appreciate you doing that. We hung out all day together.

Matthew McConaughey: (00:04)
How fun was that?

Jimmy Fallon: (00:05)
It was the greatest. We always…

Matthew McConaughey: (00:07)
Two take maximum. Two take maximum.

Jimmy Fallon: (00:09)
We nailed it, we have some good takes. That was so much fun. Happy holidays to you and your family. I’m happy to see you in person. Thanks for making the trip and being here.

Matthew McConaughey: (00:18)
Absolutely.

Jimmy Fallon: (00:19)
Do you have like family traditions? Do you have holiday plans? What do you have coming up?

Matthew McConaughey: (00:24)
This year, we’re going to get all the McConaughey family. We’re also getting Camila’s family. So when we do that and we have that multicultural Christmas, Christmas lasts a long time at our place because they celebrate on the 24th. Midnight you exchange presents. All right. So you have the big sit down dinner, 

### Titan

In [142]:
prompt_data =f"""Context: {f}

User: Execute the following actions in order
1. Summarize the conversation in 3 sentences.
2. Clasify the conversation according to the closest category within drama, comedy, talk show, news, or sports.

Summary:
"""

In [143]:
response, latency = call_bedrock('amazon.titan-tg1-large', prompt_data)
print(response, "\n\n", "Inference time:", latency)

Jimmy Fallon and Matthew McConaughey had a fun and engaging conversation about Christmas traditions, with McConaughey revealing that his family celebrates on the 24th and that he and his wife Camila use china for their sit-down dinner. They also discussed McConaughey's book, Green Lights, which was a massive hit and hit a nerve with readers. Fallon congratulated McConaughey on the success of the book and the journal he's releasing to help people write their own stories. The conversation then turned to politics, with McConaughey saying that he had considered running for governor of Texas but ultimately decided against it. Finally, they discussed McConaughey's benefit concert for those affected by the winter storm in Texas, which raised over $7.7 million.

Clasification:
The conversation falls under the category of talk show. 

 Inference time: 0:00:07.447045


### Claude

In [129]:
prompt_data =f"""
Human:
Generate a summary of the transcription in the <transcription></transcription> XML tags below.
Then, classify the transcription according to the closest category within 'drama', 'comedy', 'talk show', 'news', or 'sports'.

<transcription>
{f}
</transcription>

Assistant:
"""

In [130]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here is a summary of the transcription:

Jimmy Fallon interviews Matthew McConaughey. They discuss McConaughey's holiday plans with his family, his bestselling memoir Green Lights, his decision not to run for Texas governor, and his charity work helping Texans recover from winter storm Uri. 

Based on the content and format, I would classify this transcription as a talk show. 

 Inference time: 0:00:05.086300


In [131]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here is a summary of the transcription:

- Jimmy Fallon is interviewing actor Matthew McConaughey
- They discuss McConaughey's upcoming holiday plans with his large extended family. McConaughey says his mother insists on everyone watching each person open their presents one at a time, which takes a long time with their big family. 
- They talk about New Year's resolutions and McConaughey says his resolution last year was to work on his patience.
- Fallon congratulates McConaughey on the success of his book "Green Lights" which was a New York Times bestseller for 50 weeks and sold over 2.5 million copies. McConaughey says it hit a nerve with people and resonated with them.
- They discuss McConaughey's new companion journal to help people journal their own stories. 
- McConaughey explains his decision not to run for governor of Texas, saying at this point in his life with young children, he wants to focus on storytelling. But he does not rule out running for office in the future.
- Fall

### Jurassic-2

In [116]:
prompt_data =f"""Context: {f}

Instructions: Execute the following actions in order
1. Summarize the conversation in a few sentences.
2. Clasify the conversation according to the closest category within drama, comedy, talk show, news, or sports.

Agent:
"""

In [117]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data)
print(response, "\n\n", "Inference time:", latency)

The conversation is between Jimmy Fallon and Matthew McConaughey. They discuss McConaughey's book, Green Lights, and his family traditions for the holidays. They also discuss McConaughey's decision not to run for governor of Texas and his relief efforts for those affected by winter storms in Texas. The conversation is categorized as a talk show. 

 Inference time: 0:00:01.876295


------

## 5. Create splash pages describing upcoming events

**Use Case:** A media and entertainment company wants to create HTML pages quickly and easily, to promote their upcoming events.

**Task:** Code Generation

**Prompt Technique:** Zero-shot

### Titan

In [5]:
prompt_data ="""
There is an upcoming music concert presented by the company Music Promotions.
The event is targeting young audience in the age range between 18 and 40.
The event will be done in the Royal Music Teather.
There will be seat assignment and tickets can be bought trought the Music Promotions website.
The event is a concert of the band called Super Rockers.
The event is on June 30th 2023 and doors will be open since 20:00.

Based the this information, generate the HTML code for an attractive splash page promoting this event.
"""

In [8]:
response, latency = call_bedrock('amazon.titan-tg1-large', prompt_data)
#print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))

### Claude

In [128]:
prompt_data ="""
There is an upcoming music concert presented by the company Music Promotions.
The event is targeting young audience in the age range between 18 and 40.
The event will be done in the Royal Music Teather.
There will be seat assignment and tickets can be bought trought the Music Promotions website.
The event is a concert of the band called Super Rockers.
The event is on June 30th 2023 and doors will be open since 20:00.

Based the this information, generate the HTML code for an attractive splash page promoting this event.
"""

In [9]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
#print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))

In [10]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
#print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))

### Jurassic-2

In [11]:
prompt_data ="""
There is an upcoming music concert presented by the company Music Promotions.
The event is targeting young audience in the age range between 18 and 40.
The event will be done in the Royal Music Teather.
There will be seat assignment and tickets can be bought trought the Music Promotions website.
The event is a concert of the band called Super Rockers.
The event is on June 30th 2023 and doors will be open since 20:00.

Based the this information, generate the HTML code for an attractive splash page promoting this event.
"""

In [12]:
response, latency = call_bedrock('ai21.j2-grande-instruct', prompt_data)
#print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))

------

In [ ]:
from datetime import datetime

body = json.dumps({
    "prompt": "Human: Hi \n Assistant:",
    "max_tokens_to_sample": 4096,
    "stop_sequences":["Human:"],
    "temperature":0,
    "top_p":0.9
})

for i in range(100):
    response = bedrock.invoke_model(
    body=body,
    modelId='anthropic.claude-v1'
    )
    response_body = json.loads(response.get('body').read())
    response = response_body.get('completion')
    print(i, datetime.now(), response)


0 2023-07-15 09:46:31.561126  Hello! My name is Claude.
1 2023-07-15 09:46:32.123697  Hello! My name is Claude.
2 2023-07-15 09:46:32.715046  Hello! My name is Claude.
